In [1]:
#pip install ee

In [2]:
#pip install geemap

# Import Libraries

In [1]:
import ee
import geemap
import pandas as pd
import numpy as np 
from matplotlib import pyplot
from io import StringIO

#ee.Authenticate()  # Only needed for the first time
ee.Initialize()
Map= geemap.Map()

In [2]:
# DOC DATA 
data_doc = pd.read_excel('LTM_Data_2023_3_9.xlsx', usecols=['DOC_MG_L','SITE_ID', 'DATE_SMP'])

# Read the "Site Information" Excel sheet to get the mapping between lake names and SITE_ID
site_info_df = pd.read_excel('Site_Information_2022_8_1 (2).xlsx', usecols=['SITE_ID', 'PROGRAM_ID', 'LAKE_DEPTH_MEAN', 'SITE_NAME'])


# PROF. Coordinates 
data_path = 'ALTM-50-stations.xlsx'
df_lake_info = pd.read_excel(data_path, sheet_name='updated station coordinates', usecols=['SITE_ID', 'SITE_NAME','LATDD_CENT', 'LONDD_CENT'])


merge_data = pd.merge(data_doc, site_info_df, on='SITE_ID')
merge_data = pd.merge(merge_data, df_lake_info, on= ['SITE_ID', 'SITE_NAME'])

data_modify = merge_data[merge_data['PROGRAM_ID']=='LTM_ALTM']

In [3]:
data_modify

,SITE_ID,DATE_SMP,DOC_MG_L,PROGRAM_ID,SITE_NAME,LAKE_DEPTH_MEAN,LATDD_CENT,LONDD_CENT
0,040850,1997-09-22,3.100000,LTM_ALTM,Squaw Lake,3.4,43.63276,-74.73863
2,040850,1995-03-02,3.031000,LTM_ALTM,Squaw Lake,3.4,43.63276,-74.73863
4,040850,1996-08-19,3.595000,LTM_ALTM,Squaw Lake,3.4,43.63276,-74.73863
6,040850,2003-11-06,4.232000,LTM_ALTM,Squaw Lake,3.4,43.63276,-74.73863
8,040850,2005-07-11,3.344000,LTM_ALTM,Squaw Lake,3.4,43.63276,-74.73863
...,...,...,...,...,...,...,...,...
17863,1A3-048,2011-05-18,4.283944,LTM_ALTM,Grass Pond,1.5,43.69207,-75.06172
17864,1A3-048,2015-06-10,5.509394,LTM_ALTM,Grass Pond,1.5,43.69207,-75.06172
17865,1A3-048,2016-07-14,4.884900,LTM_ALTM,Grass Pond,1.5,43.69207,-75.06172
17866,1A3-048,2016-09-15,6.344000,LTM_ALTM,Grass Pond,1.5,43.69207,-75.06172


In [7]:
 s2_new = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filter(ee.Filter.calendarRange(2019, 2023, 'year')) \
        .filterBounds(lake) \
        .filter(ee.Filter.lt('CLOUD_COVER', 15)) \
        .map(maskS2clouds) \
        .map(normalizedBands)\
        .select(new_bands,STD_NEW)

In [10]:
s2_image = s2_new.first()  # Get the first image from the s2_new collection
band_names = s2_image.bandNames().getInfo()  # Get the band names of the image
print(band_names)  # Print the band names


EEException: Image.bandNames: Parameter 'image' is required.

# REFLECTANCE data prep

In [4]:
import ee
import geemap
import pandas as pd
import numpy as np 
from matplotlib import pyplot
from io import StringIO


Map= geemap.Map()

# Authenticate and initialize the Earth Engine API
#ee.Authenticate()  # Only needed for the first time
ee.Initialize()

# Create a list of lake names
lake_names = [
    'Little Hope Pond', 'Big Hope Pond', 'East Copperas Pond', 'Sunday Pond', 'Sochia Pond',
    'Grass Pond (3)', 'Little Clear Pond', 'Loon Hollow Pond', 'Willys Lake', 'Woods Lake',
    'Middle Settlement Lake', 'Middle Branch Lake', 'Limekiln Lake', 'Squaw Lake', 'Indian Lake',
    'Brook Trout Lake', 'Lost Pond', 'North Lake', 'Willis Lake', 'Long Pond', 'Carry Pond',
    'Lake Colden', 'Avalanche Lake', 'Little Simon Pond', 'Raquette Lake Reservoir', 'G Lake',
    'Constable Pond', 'Middle Pond', 'Arbutus Pond', 'Sagamore Lake', 'Black Pond', 'Windfall Pond',
    'Queer Lake', 'Heart Lake', 'Big Moose Lake', 'Cascade Lake', 'Dart Lake', 'Little Echo Pond',
    'Moss Lake', 'Lake Rondaxe', 'Squash Pond', 'West Pond', 'Owen Pond', 'Jockeybush Lake',
    'Barnes Lake', 'Clear Pond', 'Otter Lake', 'Nate Pond', 'Grass Pond', 'South Lake (East Branch)'
]

# Define the Landsat 5 bands and their corresponding standard names
# LC5_BANDS = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'ST_B6', 'QA_PIXEL']
# STD_NAMES = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'temp', 'QA']

sentinel2_bands =['B1','B2','B3','B4','B5','B6','QA60','B8A','B11']
STD_NAMES = ['Aerosols', 'Blue', 'Green', 'Red', 'RedEdge1','RedEdge2','Cloud','NIR','SWIR1']


def normalizedBands(image):
    # Calculate the differences between NIR and other bands
    nir = image.select('B8A').rename('NIR')
    blue_nir_diff = nir.subtract(image.select('B2')).rename('blue_nir_diff')
    green_nir_diff = nir.subtract(image.select('B3')).rename('green_nir_diff')
    red_nir_diff = nir.subtract(image.select('B4')).rename('red_nir_diff')
    lake_label = nir.multiply(0).add(34).rename('LAKE_LABEL')

    return image.addBands(blue_nir_diff, None, True).addBands(green_nir_diff, None, True).addBands(red_nir_diff, None, True).addBands(lake_label, None, True)

new_bands= ['blue_nir_diff','green_nir_diff','red_nir_diff','NIR','LAKE_LABEL','B5','B6']
STD_NEW= ['blue_nir_diff','green_nir_diff','red_nir_diff','NIR','LAKE_LABEL','RedEdge1','RedEdge2']


# Define the function to mask clouds and cloud shadows from Landsat 5 surface reflectance imagery
def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Create the cloud mask
    cloudMask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Apply the cloud mask to the image
    maskedImage = image.updateMask(cloudMask)

    return maskedImage


# Define the function to compute the mean reflectance values for the specified bands within the region of interest (lake)
def reflectance(img, lake):
    reflectance_values = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=lake, scale=30).select(STD_NEW)
    return img.set('DATE_SMP', img.date().format()).set('reflectance', reflectance_values)

# Initialize an empty list to store the dataframes for each lake
dfs = []

# Create a dictionary to map each lake name to a numerical label
lake_name_to_label = {lake_name: i for i, lake_name in enumerate(lake_names)}

# Loop through each lake name and retrieve Landsat 5 imagery for that lake
for SITE_NAME in lake_names:
    # Retrieve Landsat 5 imagery for the specific lake
    lake = ee.FeatureCollection('projects/ee-touhedakhanom14/assets/stations-coord')\
        .filter(ee.Filter.eq('SITE_NAME', SITE_NAME))
    
    s2_new = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filter(ee.Filter.calendarRange(2019, 2023, 'year')) \
        .filterBounds(lake) \
        .filter(ee.Filter.lt('CLOUD_COVER', 15)) \
        .map(maskS2clouds) \
        .map(normalizedBands)\
        .select(new_bands,STD_NEW)
    
   # Map.addLayer(lake, {'color': 'red'})  # Adding the original point to show the difference
    
#     s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
#     .filter(ee.Filter.calendarRange(2019, 2023, 'year')) \
#     .map(maskS2clouds) \
#     .select(sentinel2_bands, STD_NAMES)\
#     .filterBounds(lake) \
#     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))

    # Map the reflectance function over the Landsat 5 ImageCollection for the specific lake
    map_reflectance = s2_new.map(lambda img: reflectance(img, lake))

    # Reduce the mapped image collection to get reflectance values for the specific lake
    list_reflectance = map_reflectance.reduceColumns(ee.Reducer.toList(2), ['DATE_SMP', 'reflectance']).values().get(0)

    # Convert the results to a pandas DataFrame
    df_reflectance = pd.DataFrame(list_reflectance.getInfo(), columns=['DATE_SMP', 'reflectance'])
    df_reflectance['DATE_SMP'] = pd.to_datetime(df_reflectance['DATE_SMP'])
    df_reflectance['DATE_SMP'] = df_reflectance['DATE_SMP'].dt.date
    df_reflectance['reflectance'] = df_reflectance['reflectance'].apply(lambda x: {k: v/10000 for k, v in x.items() if v is not None})

    # Unpack the 'reflectance' dictionary and create separate columns for each band
    df_reflectance = pd.concat([df_reflectance.drop('reflectance', axis=1),
                                df_reflectance['reflectance'].apply(pd.Series).astype('float64', errors='ignore')], axis=1)

    # Add a new column for the lake name
    df_reflectance['SITE_NAME'] = SITE_NAME
    
    # Add a new column for the lake label (numeric representation)
    df_reflectance['LAKE_LABEL'] = lake_name_to_label[SITE_NAME]

    # Add the DataFrame to the list
    dfs.append(df_reflectance)

# Concatenate all DataFrames into a single DataFrame
df_all_lakes = pd.concat(dfs, ignore_index=True)

# Sort the DataFrame by 'DATE_SMP' in ascending order
df_all_lakes.sort_values(by='DATE_SMP', inplace=True)


# Map.centerObject(lake, 10)
# Map
# Display the DataFrame containing reflectance values for all lakes with separate columns for each lake
# print(df_all_lakes)


In [5]:
df_all_lakes

,DATE_SMP,reflectance,SITE_NAME,LAKE_LABEL


In [26]:
df_all_lakes.dropna(inplace=True)


In [27]:
df_all_lakes

,DATE_SMP,reflectance,SITE_NAME,LAKE_LABEL


In [28]:
pd.set_option('display.max_rows', None)
print(df_all_lakes)

Empty DataFrame
Columns: [DATE_SMP, reflectance, SITE_NAME, LAKE_LABEL]
Index: []


In [12]:
df_all_lakes['DATE_SMP'] = pd.to_datetime(df_all_lakes['DATE_SMP'])
data_modify['DATE_SMP'] = pd.to_datetime(data_modify['DATE_SMP'])

In [13]:
data_modify.sort_values('DATE_SMP', inplace=True)
df_all_lakes.sort_values('DATE_SMP', inplace=True)


In [14]:
# Create the 3-day time window
window_size = pd.Timedelta(days=3)

# Merge data within the 3-day time window
merged_data = pd.merge_asof(data_modify, df_all_lakes, on='DATE_SMP', by='SITE_NAME', tolerance=window_size)
merged_data = merged_data.dropna(subset=['Blue'])
merged_data = merged_data.dropna(subset=['LAKE_DEPTH_MEAN'])
merged_data

KeyError: ['Blue']

# Error

In [ ]:
# num_rows, num_cols = X.shape
# print("Number of rows:", num_rows)
# print("Number of columns:", num_cols)

In [ ]:
#pip install xgboost


In [ ]:
#pip install lightgbm

In [ ]:
# # Preparing the training data
# X = merged_data[['LAKE_LABEL','Aerosols', 'Blue', 'Green', 'Red', 'RedEdge1','RedEdge2','Cloud','NIR','SWIR1','LAKE_DEPTH_MEAN']]  # Features
# y = merged_data['DOC_MG_L']  # Target variable


# WITHOUT LAKE LABEL AS FEATURES 3 day window/n

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# Preparing the training data
X = merged_data[['Blue', 'Green', 'Red','LAKE_DEPTH_MEAN']]  # Features
y = merged_data['DOC_MG_L']  # Target variable

# Group the data by 'LAKE_LABEL'
grouped_data = merged_data.groupby('LAKE_LABEL')

# Initialize lists to store the training and testing data for all lakes
X_train_all, X_test_all, y_train_all, y_test_all = [], [], [], []

# Loop through each lake group and split data into training and testing sets
for _, lake_group in grouped_data:
    if len(lake_group) < 2:
        # Skip this lake group as it has insufficient data for splitting
        continue
    X_lake = lake_group[['Blue', 'Green', 'Red','LAKE_DEPTH_MEAN']]
    y_lake = lake_group['DOC_MG_L']
   
    # Split data into training and testing sets for the current lake
    X_train_lake, X_test_lake, y_train_lake, y_test_lake = train_test_split(X_lake, y_lake, test_size=0.3, random_state=42)
   
    # Append the data for the current lake to the corresponding lists
    X_train_all.append(X_train_lake)
    X_test_all.append(X_test_lake)
    y_train_all.append(y_train_lake)
    y_test_all.append(y_test_lake)

# Concatenate the training and testing data for all lakes
X_train = pd.concat(X_train_all)
X_test = pd.concat(X_test_all)
y_train = pd.concat(y_train_all)
y_test = pd.concat(y_test_all)

print('**************Data Points Used***************************')
num_rows, num_cols = X.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_cols)

# Initialize models
models = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'SVR': SVR(),
    'MLPRegressor': MLPRegressor(random_state=42)
}

# Initialize lists to store model evaluation results
model_names, r2_scores, mses, rmses, maes = [], [], [], [], []

# Train and evaluate each model
for model_name, model in models.items():
    # Fit the model
    model.fit(X_train, y_train)

    # Predict DOC values for the testing data
    y_pred = model.predict(X_test)

    # Create DataFrames for the training and testing data with all bands and the predicted values
    data_train = X_train.copy()
    data_train['original_DOC'] = y_train
    data_train['predicted_DOC'] = model.predict(X_train)

    data_test = X_test.copy()
    data_test['original_DOC'] = y_test
    data_test['predicted_DOC'] = y_pred

    # Calculate evaluation metrics
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)

    # Append the results to the lists
    model_names.append(model_name)
    r2_scores.append(r2)
    mses.append(mse)
    rmses.append(rmse)
    maes.append(mae)

# Display the predicted DOC values for the training and testing data
print(f'*************Predicted Training data ({model_name})***************')
print(data_train)

print(f'*************Predicted Testing data ({model_name})***************')
print(data_test)

# Create a dataframe to store the results
evaluation_df_WITHOUT3 = pd.DataFrame({
    'Model': model_names,
    'R^2': r2_scores,
    'MSE': mses,
    'RMSE': rmses,
    'MAE': maes
})

# Print the dataframe
print('# WITHOUT LAKE LABEL AS FEATURES 3 day window/n')
print(evaluation_df_WITHOUT3)

In [ ]:
#evaluation_df_WITHOUT3.to_excel(r'/Users/touhedakhanom/Downloads/NSF SUMMER Research /Research /Machine Learning/ML/EXCEL SURFACE REFLECTANCE\SR WITHOUT LAKE LABEL AS FEATURES 3 day window.xlsx', index=False)


# WITH LAKE FEATURE AS 3 DAY WINDOW

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler


# Preparing the training data
X = merged_data[['LAKE_LABEL','Blue', 'Green', 'Red','LAKE_DEPTH_MEAN']]  # Features
y = merged_data['DOC_MG_L']  # Target variable

# Group the data by 'LAKE_LABEL'
grouped_data = merged_data.groupby('LAKE_LABEL')

# Initialize lists to store the training and testing data for all lakes
X_train_all, X_test_all, y_train_all, y_test_all = [], [], [], []

# Loop through each lake group and split data into training and testing sets
for _, lake_group in grouped_data:
    if len(lake_group) < 2:
        # Skip this lake group as it has insufficient data for splitting
        continue
    X_lake = lake_group[['LAKE_LABEL','Blue', 'Green', 'Red', 'LAKE_DEPTH_MEAN']]
    y_lake = lake_group['DOC_MG_L']
   
    # Split data into training and testing sets for the current lake
    X_train_lake, X_test_lake, y_train_lake, y_test_lake = train_test_split(X_lake, y_lake, test_size=0.3, random_state=42)
   
    # Append the data for the current lake to the corresponding lists
    X_train_all.append(X_train_lake)
    X_test_all.append(X_test_lake)
    y_train_all.append(y_train_lake)
    y_test_all.append(y_test_lake)

# Concatenate the training and testing data for all lakes
X_train = pd.concat(X_train_all)
X_test = pd.concat(X_test_all)
y_train = pd.concat(y_train_all)
y_test = pd.concat(y_test_all)




print('**************Data Points Used***************************')
num_rows, num_cols = X.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_cols)



# Initialize models
models = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'SVR': SVR(),
    'MLPRegressor': MLPRegressor(random_state=42)
}

# Initialize lists to store model evaluation results
model_names, r2_scores, mses, rmses, maes = [], [], [], [], []

# Train and evaluate each model
for model_name, model in models.items():
    # Fit the model
    model.fit(X_train, y_train)

    # Predict DOC values for the testing data
    y_pred = model.predict(X_test)

    # Create DataFrames for the training and testing data with all bands and the predicted values
    data_train = X_train.copy()
    data_train['original_DOC'] = y_train
    data_train['predicted_DOC'] = model.predict(X_train)

    data_test = X_test.copy()
    data_test['original_DOC'] = y_test
    data_test['predicted_DOC'] = y_pred

    # Calculate evaluation metrics
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)

    # Append the results to the lists
    model_names.append(model_name)
    r2_scores.append(r2)
    mses.append(mse)
    rmses.append(rmse)
    maes.append(mae)

# Display the predicted DOC values for the training and testing data
print(f'*************Predicted Training data ({model_name})***************')
print(data_train)

print(f'*************Predicted Testing data ({model_name})***************')
print(data_test)

# Create a dataframe to store the results
evaluation_df_WITH3 = pd.DataFrame({
    'Model': model_names,
    'R^2': r2_scores,
    'MSE': mses,
    'RMSE': rmses,
    'MAE': maes
})

# Print the dataframe
print('# WITH LAKE LABEL AS FEATURES 3 day window/n')
print(evaluation_df_WITH3)

In [ ]:
# Plot the scatter plot for all lakes combined
plt.scatter( data_train['original_DOC'],data_train['predicted_DOC'], color='blue', label='Training Data')
plt.scatter(data_test['original_DOC'],data_test['predicted_DOC'],  color='red', label='Testing Data')

# Set plot title and labels
plt.title('Scatter Plot for All Lakes')
plt.xlabel('Original DOC')
plt.ylabel('Predicted DOC')
plt.legend()

# Show the plot
plt.show()

In [ ]:
#evaluation_df_WITH3.to_excel(r'/Users/touhedakhanom/Downloads/NSF SUMMER Research /Research /Machine Learning/ML/EXCEL SURFACE REFLECTANCE\SR WITH LAKE LABEL AS FEATURES 3 day window.xlsx', index=False)


# WITHOUT LAKE LABEL AS FEATURES 1 day window/n


In [ ]:
# Create the 3-day time window
window_size = pd.Timedelta(days=1)

# Merge data within the 3-day time window
merged_data_1 = pd.merge_asof(data_modify, df_all_lakes, on='DATE_SMP', by='SITE_NAME', tolerance=window_size)
merged_data_1 = merged_data_1.dropna(subset=['Blue'])
merged_data_1 = merged_data_1.dropna(subset=['LAKE_DEPTH_MEAN'])
merged_data_1

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# Preparing the training data
X = merged_data_1[['Blue', 'Green', 'Red','LAKE_DEPTH_MEAN']]  # Features
y = merged_data_1['DOC_MG_L']  # Target variable

# Group the data by 'LAKE_LABEL'
grouped_data = merged_data_1.groupby('LAKE_LABEL')

# Initialize lists to store the training and testing data for all lakes
X_train_all, X_test_all, y_train_all, y_test_all = [], [], [], []

# Loop through each lake group and split data into training and testing sets
for _, lake_group in grouped_data:
    if len(lake_group) < 4: #Give me 0.3 but this is better 
        # Skip this lake group as it has insufficient data for splitting
        continue
    X_lake = lake_group[['Blue', 'Green', 'Red','LAKE_DEPTH_MEAN']]
    y_lake = lake_group['DOC_MG_L']
   
    # Split data into training and testing sets for the current lake
    X_train_lake, X_test_lake, y_train_lake, y_test_lake = train_test_split(X_lake, y_lake, test_size=0.3, random_state=42)
   
    # Append the data for the current lake to the corresponding lists
    X_train_all.append(X_train_lake)
    X_test_all.append(X_test_lake)
    y_train_all.append(y_train_lake)
    y_test_all.append(y_test_lake)

# Concatenate the training and testing data for all lakes
X_train = pd.concat(X_train_all)
X_test = pd.concat(X_test_all)
y_train = pd.concat(y_train_all)
y_test = pd.concat(y_test_all)

print('**************Data Points Used***************************')
num_rows, num_cols = X.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_cols)

# Initialize models
models = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'SVR': SVR(),
    'MLPRegressor': MLPRegressor(random_state=42)
}

# Initialize lists to store model evaluation results
model_names, r2_scores, mses, rmses, maes = [], [], [], [], []

# Train and evaluate each model
for model_name, model in models.items():
    # Fit the model
    model.fit(X_train, y_train)

    # Predict DOC values for the testing data
    y_pred = model.predict(X_test)

    # Create DataFrames for the training and testing data with all bands and the predicted values
    data_train = X_train.copy()
    data_train['original_DOC'] = y_train
    data_train['predicted_DOC'] = model.predict(X_train)

    data_test = X_test.copy()
    data_test['original_DOC'] = y_test
    data_test['predicted_DOC'] = y_pred

    # Calculate evaluation metrics
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)

    # Append the results to the lists
    model_names.append(model_name)
    r2_scores.append(r2)
    mses.append(mse)
    rmses.append(rmse)
    maes.append(mae)

# Display the predicted DOC values for the training and testing data
print(f'*************Predicted Training data ({model_name})***************')
print(data_train)

print(f'*************Predicted Testing data ({model_name})***************')
print(data_test)

# Create a dataframe to store the results
evaluation_df_WITHOUT1 = pd.DataFrame({
    'Model': model_names,
    'R^2': r2_scores,
    'MSE': mses,
    'RMSE': rmses,
    'MAE': maes
})

# Print the dataframe
print('# WITHOUT LAKE LABEL AS FEATURES 1 day window/n')
print(evaluation_df_WITHOUT1)

In [ ]:
#evaluation_df_WITHOUT1.to_excel(r'/Users/touhedakhanom/Downloads/NSF SUMMER Research /Research /Machine Learning/ML/EXCEL SURFACE REFLECTANCE\SR WITHOUT LAKE LABEL AS FEATURES 1 day window.xlsx', index=False)


# WITH LAKE LABEL AS FEATURES 1 day window/n


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler


# Preparing the training data
X = merged_data_1[['LAKE_LABEL','Blue', 'Green', 'Red','LAKE_DEPTH_MEAN']]  # Features
y = merged_data_1['DOC_MG_L']  # Target variable

# Group the data by 'LAKE_LABEL'
grouped_data = merged_data_1.groupby('LAKE_LABEL')

# Initialize lists to store the training and testing data for all lakes
X_train_all, X_test_all, y_train_all, y_test_all = [], [], [], []

# Loop through each lake group and split data into training and testing sets
for _, lake_group in grouped_data:
    if len(lake_group) < 4:
        # Skip this lake group as it has insufficient data for splitting
        continue
    X_lake = lake_group[['LAKE_LABEL','Blue', 'Green', 'Red', 'LAKE_DEPTH_MEAN']]
    y_lake = lake_group['DOC_MG_L']
   
    # Split data into training and testing sets for the current lake
    X_train_lake, X_test_lake, y_train_lake, y_test_lake = train_test_split(X_lake, y_lake, test_size=0.3, random_state=42)
   
    # Append the data for the current lake to the corresponding lists
    X_train_all.append(X_train_lake)
    X_test_all.append(X_test_lake)
    y_train_all.append(y_train_lake)
    y_test_all.append(y_test_lake)

# Concatenate the training and testing data for all lakes
X_train = pd.concat(X_train_all)
X_test = pd.concat(X_test_all)
y_train = pd.concat(y_train_all)
y_test = pd.concat(y_test_all)




print('**************Data Points Used***************************')
num_rows, num_cols = X.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_cols)



# Initialize models
models = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'SVR': SVR(),
    'MLPRegressor': MLPRegressor(random_state=42)
}

# Initialize lists to store model evaluation results
model_names, r2_scores, mses, rmses, maes = [], [], [], [], []

# Train and evaluate each model
for model_name, model in models.items():
    # Fit the model
    model.fit(X_train, y_train)

    # Predict DOC values for the testing data
    y_pred = model.predict(X_test)

    # Create DataFrames for the training and testing data with all bands and the predicted values
    data_train = X_train.copy()
    data_train['original_DOC'] = y_train
    data_train['predicted_DOC'] = model.predict(X_train)

    data_test = X_test.copy()
    data_test['original_DOC'] = y_test
    data_test['predicted_DOC'] = y_pred

    # Calculate evaluation metrics
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)

    # Append the results to the lists
    model_names.append(model_name)
    r2_scores.append(r2)
    mses.append(mse)
    rmses.append(rmse)
    maes.append(mae)

# Display the predicted DOC values for the training and testing data
print(f'*************Predicted Training data ({model_name})***************')
print(data_train)

print(f'*************Predicted Testing data ({model_name})***************')
print(data_test)

# Create a dataframe to store the results
evaluation_df_WITH1 = pd.DataFrame({
    'Model': model_names,
    'R^2': r2_scores,
    'MSE': mses,
    'RMSE': rmses,
    'MAE': maes
})

# Print the dataframe
print('# WITH LAKE LABEL AS FEATURES 1 day window/n')
print(evaluation_df_WITH1)

In [ ]:
#evaluation_df_WITH1.to_excel(r'/Users/touhedakhanom/Downloads/NSF SUMMER Research /Research /Machine Learning/ML/EXCEL SURFACE REFLECTANCE\SR WITH LAKE LABEL AS FEATURES 1 day window.xlsx', index=False)


# WITHOUT LAKE LABEL AS FEATURES 7 day window/n


In [ ]:
# Create the 3-day time window
window_size = pd.Timedelta(days=7)

# Merge data within the 3-day time window
merged_data_7 = pd.merge_asof(data_modify, df_all_lakes, on='DATE_SMP', by='SITE_NAME', tolerance=window_size)
merged_data_7 = merged_data_7.dropna(subset=['Blue'])
merged_data_7 = merged_data_7.dropna(subset=['LAKE_DEPTH_MEAN'])
merged_data_7

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# Preparing the training data
X = merged_data_7[['Blue', 'Green', 'Red','LAKE_DEPTH_MEAN']]  # Features
y = merged_data_7['DOC_MG_L']  # Target variable

# Group the data by 'LAKE_LABEL'
grouped_data = merged_data_7.groupby('LAKE_LABEL')

# Initialize lists to store the training and testing data for all lakes
X_train_all, X_test_all, y_train_all, y_test_all = [], [], [], []

# Loop through each lake group and split data into training and testing sets
for _, lake_group in grouped_data:
    if len(lake_group) < 4:
        # Skip this lake group as it has insufficient data for splitting
        continue
    X_lake = lake_group[['Blue', 'Green', 'Red','LAKE_DEPTH_MEAN']]
    y_lake = lake_group['DOC_MG_L']
   
    # Split data into training and testing sets for the current lake
    X_train_lake, X_test_lake, y_train_lake, y_test_lake = train_test_split(X_lake, y_lake, test_size=0.3, random_state=42)
   
    # Append the data for the current lake to the corresponding lists
    X_train_all.append(X_train_lake)
    X_test_all.append(X_test_lake)
    y_train_all.append(y_train_lake)
    y_test_all.append(y_test_lake)

# Concatenate the training and testing data for all lakes
X_train = pd.concat(X_train_all)
X_test = pd.concat(X_test_all)
y_train = pd.concat(y_train_all)
y_test = pd.concat(y_test_all)

print('**************Data Points Used***************************')
num_rows, num_cols = X.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_cols)

# Initialize models
models = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'SVR': SVR(),
    'MLPRegressor': MLPRegressor(random_state=42)
}

# Initialize lists to store model evaluation results
model_names, r2_scores, mses, rmses, maes = [], [], [], [], []

# Train and evaluate each model
for model_name, model in models.items():
    # Fit the model
    model.fit(X_train, y_train)

    # Predict DOC values for the testing data
    y_pred = model.predict(X_test)

    # Create DataFrames for the training and testing data with all bands and the predicted values
    data_train = X_train.copy()
    data_train['original_DOC'] = y_train
    data_train['predicted_DOC'] = model.predict(X_train)

    data_test = X_test.copy()
    data_test['original_DOC'] = y_test
    data_test['predicted_DOC'] = y_pred

    # Calculate evaluation metrics
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)

    # Append the results to the lists
    model_names.append(model_name)
    r2_scores.append(r2)
    mses.append(mse)
    rmses.append(rmse)
    maes.append(mae)

# Display the predicted DOC values for the training and testing data
print(f'*************Predicted Training data ({model_name})***************')
print(data_train)

print(f'*************Predicted Testing data ({model_name})***************')
print(data_test)

# Create a dataframe to store the results
evaluation_df_WITHOUT7 = pd.DataFrame({
    'Model': model_names,
    'R^2': r2_scores,
    'MSE': mses,
    'RMSE': rmses,
    'MAE': maes
})

# Print the dataframe
print('# WITHOUT LAKE LABEL AS FEATURES 7 day window/n')
print(evaluation_df_WITHOUT7)

In [ ]:
evaluation_df_WITHOUT7.to_excel(r'/Users/touhedakhanom/Downloads/NSF SUMMER Research /Research /Machine Learning/ML/EXCEL SURFACE REFLECTANCE\SR WITHOUT LAKE LABEL AS FEATURES 7 day window.xlsx', index=False)


# WITH LAKE LABEL AS FEATURES 7 day window/n


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler


# Preparing the training data
X = merged_data_7[['LAKE_LABEL', 'Aerosols', 'Blue', 'Green', 'Red', 'RedEdge1', 'RedEdge2', 'Cloud', 'NIR', 'SWIR1', 'LAKE_DEPTH_MEAN']]  # Features
y = merged_data_7['DOC_MG_L']  # Target variable

# Group the data by 'LAKE_LABEL'
grouped_data = merged_data_7.groupby('LAKE_LABEL')

# Initialize lists to store the training and testing data for all lakes
X_train_all, X_test_all, y_train_all, y_test_all = [], [], [], []

# Loop through each lake group and split data into training and testing sets
for _, lake_group in grouped_data:
    if len(lake_group) < 4:
        # Skip this lake group as it has insufficient data for splitting
        continue
    X_lake = lake_group[['LAKE_LABEL', 'Aerosols', 'Blue', 'Green', 'Red', 'RedEdge1', 'RedEdge2', 'Cloud', 'NIR', 'SWIR1', 'LAKE_DEPTH_MEAN']]
    y_lake = lake_group['DOC_MG_L']
   
    # Split data into training and testing sets for the current lake
    X_train_lake, X_test_lake, y_train_lake, y_test_lake = train_test_split(X_lake, y_lake, test_size=0.3, random_state=42)
   
    # Append the data for the current lake to the corresponding lists
    X_train_all.append(X_train_lake)
    X_test_all.append(X_test_lake)
    y_train_all.append(y_train_lake)
    y_test_all.append(y_test_lake)

# Concatenate the training and testing data for all lakes
X_train = pd.concat(X_train_all)
X_test = pd.concat(X_test_all)
y_train = pd.concat(y_train_all)
y_test = pd.concat(y_test_all)




print('**************Data Points Used***************************')
num_rows, num_cols = X.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_cols)



# Initialize models
models = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'SVR': SVR(),
    'MLPRegressor': MLPRegressor(random_state=42)
}

# Initialize lists to store model evaluation results
model_names, r2_scores, mses, rmses, maes = [], [], [], [], []

# Train and evaluate each model
for model_name, model in models.items():
    # Fit the model
    model.fit(X_train, y_train)

    # Predict DOC values for the testing data
    y_pred = model.predict(X_test)

    # Create DataFrames for the training and testing data with all bands and the predicted values
    data_train = X_train.copy()
    data_train['original_DOC'] = y_train
    data_train['predicted_DOC'] = model.predict(X_train)

    data_test = X_test.copy()
    data_test['original_DOC'] = y_test
    data_test['predicted_DOC'] = y_pred

    # Calculate evaluation metrics
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)

    # Append the results to the lists
    model_names.append(model_name)
    r2_scores.append(r2)
    mses.append(mse)
    rmses.append(rmse)
    maes.append(mae)

# Display the predicted DOC values for the training and testing data
print(f'*************Predicted Training data ({model_name})***************')
print(data_train)

print(f'*************Predicted Testing data ({model_name})***************')
print(data_test)

# Create a dataframe to store the results
evaluation_df_WITH7 = pd.DataFrame({
    'Model': model_names,
    'R^2': r2_scores,
    'MSE': mses,
    'RMSE': rmses,
    'MAE': maes
})

# Print the dataframe
print('# WITH LAKE LABEL AS FEATURES 7 day window/n')
print(evaluation_df_WITH7)

In [ ]:
#evaluation_df_WITH7.to_excel(r'/Users/touhedakhanom/Downloads/NSF SUMMER Research /Research /Machine Learning/ML\SR WITH LAKE LABEL AS FEATURES 7 day window.xlsx', index=False)


In [ ]:
# Plot the scatter plot for all lakes combined
plt.scatter( data_train['original_DOC'],data_train['predicted_DOC'], color='blue', label='Training Data')
plt.scatter(data_test['original_DOC'],data_test['predicted_DOC'],  color='red', label='Testing Data')

# Set plot title and labels
plt.title('Scatter Plot for All Lakes')
plt.xlabel('Original DOC')
plt.ylabel('Predicted DOC')
plt.legend()

# Show the plot
plt.show()